In [1]:
# Libraries:
import pandas as pd
import numpy as np
from __future__ import division
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
%pylab inline
import time


Populating the interactive namespace from numpy and matplotlib


In [2]:
# Config:
# In order to display all the columns:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 50

### FUNCTIONS

In [3]:
# Functions to put data in the correct format for estimation:
def get_set_tags(dataframe):
    # Use this function to get the set of tags in the document:
    user_tags=dataframe.usertag
    list_tags=[]
    for i in user_tags:
        for j in i.split(','):
            list_tags.append(j)
    set_tags=list(set(list_tags))
    return set_tags
# removing the null tags:
def remove_objects(list_values,obj="null"):
    return [x for x in list_values if x != obj]

# Functions to separate the usertag columns:
def create_tag_index_dictio(training_df,column="usertag",split_char=","):
    # Creates a dictionary with key: tag and value: list of rows
    tag_series=training_df[column].apply(lambda x:x.split(split_char))
    k=0
    tag_index={}
    for tags_line,index_tag in zip(tag_series,tag_series.index):
        k+=1
        for tag in tags_line:
            if tag != "null":
                if tag not in tag_index.keys():
                    tag_index[tag]=[index_tag]
                else:
                    tag_index[tag].append(index_tag)
        if k%500000==0:
            print "%d completed"%(k)
    return tag_index

def get_tagsDF(dictio_tag_index,training_set):
    # Create the dictionary with key:usertag, value: list with 1 tha impression correspond to this tag 
    N=training_set.shape[0]
    dictio_tags_values={}
    for user_tag in dictio_tag_index.keys():
        sparce_array=[0]*N
        for i in dictio_tag_index[user_tag]:
            sparce_array[i]=1
        dictio_tags_values[user_tag]=sparce_array
    print "%d users completed, creating dataframe"%(len(dictio_tags_values.keys()))
    return pd.DataFrame(dictio_tags_values)


### Importing datasets:

In [4]:
# Training set:
start_time=time.time()
filepath="../dataset/train.csv" 
#filepath="../dataset/validation.csv"
data=pd.read_csv(filepath)
print "data imported, num rows: %d, num cols: %d"%(data.shape[0],data.shape[1])


data imported, num rows: 2697738, num cols: 26


In [5]:
# validation set:
#filepath_val="../dataset/validation.csv"
#data_val=pd.read_csv(filepath_val)

#validation_set=data_val[data_val["bidprice"]>data_val["payprice"]]
#validation_set=validation_set.reset_index(drop=True)
#print "validation set: %d rows"%(validation_set.shape[0])

In [6]:
training_set=data[data["bidprice"]>data["payprice"]]
training_set=training_set.reset_index(drop=True)
print "training set: %d rows"%(training_set.shape[0])

training set: 2661039 rows


In [7]:
# According to the iPINyou paper:
columns_to_dummy=["weekday","hour","region","city","adexchange","slotwidth","slotheight","slotvisibility",
                  "slotformat","slotprice"]
columns_to_drop=["bidid","logtype","userid","urlid","url","bidprice","payprice","keypage","creative","domain","IP","slotid"]

In [ ]:
# Droping columns that we don't use:
training_set.drop(columns_to_drop, axis=1, inplace=True)
print "remaining cols: %d" % (training_set.shape[1])

remaining cols: 14


In [ ]:
# Getting the set of tags:
list_tags=get_set_tags(training_set)
list_tags=remove_objects(list_tags)

# creating dictionaries:
dictio_tag_index=create_tag_index_dictio(training_set)
dictio_os_browser=create_tag_index_dictio(training_set,"useragent","_")

In [ ]:
# creating new dataframe:
N = training_set.shape[0]
tags_valuesDF=get_tagsDF(dictio_tag_index,training_set)
os_browserDF=get_tagsDF(dictio_os_browser,training_set)

In [ ]:
training_set=pd.get_dummies(training_set,columns=columns_to_dummy)

In [ ]:
clean_data = pd.concat([training_set, tags_valuesDF,os_browserDF], axis=1)

In [ ]:
clean_data.drop(["useragent","usertag"],axis=1,inplace=True)

In [ ]:
clean_data.head()
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
#clean_data.to_csv("pp_train_data.csv")

In [ ]:
#x=pd.read_csv("pp_train_data.csv")